# PPTXファイルの各スライドをPNGで出力するサンプル

---
**⚠️ 警告:** Databricks Serverless では実行できず、 Dedicated モードの Classic Cluster で実行する必要があります。
---

## 事前準備


- `/contents/00_setup` のノートブックを実行して、 PowerPoint のファイルを Volume に配置してください。

## 必要なライブラリのインストール

- libreoffice: pptxを開くために必要
- python3-uno: libreofficeをPythonで操作(unoconvで利用)
- unoconv: pptx→pdf
- fonts-xx: 日本語文字認識
- poppler-utils: pdf2imageで使用する

In [0]:
%sh
sudo apt update -y >/dev/null 2>&1
sudo apt install libreoffice python3-uno unoconv fonts-noto-cjk fonts-ipafont fonts-ipaexfont poppler-utils -y  >/dev/null 2>&1

In [0]:
%pip install pdf2image -q
dbutils.library.restartPython()

## 共通設定の読み取りとライブラリのインポート

In [0]:
%run ../contents/00_config

## ファイルの確認

In [0]:
# Volume に書かれたことを確認
print("-- databricks_overview_by_ai.pptx")
pptx_path = source_dir + "/landing/raw_files"
pptx_path += "/audit__ingest_timestamp=2025-10-14T10:07:33Z"
pptx_path += "/drive/root:/test/files/databricks_overview_by_ai.pptx"
display(dbutils.fs.ls(pptx_path))

In [0]:
pdf_output_dir = source_dir + "/tips/02/pdf"
pdf_output_path = pdf_output_dir + "/databricks_overview_by_ai.pdf"
png_output_dir = source_dir + "/tips/02/png"

In [0]:
dbutils.fs.mkdirs(pdf_output_dir)
dbutils.fs.ls(pdf_output_dir)

In [0]:
dbutils.fs.mkdirs(png_output_dir)
dbutils.fs.ls(png_output_dir)

## PPTX→PDF

In [0]:
workdir = "/tmp/unoconv_work"
tmp_outdir = "/tmp/unoconv_out"

In [0]:
import os, shutil, subprocess, pathlib, shlex

# 作業ディレクトリを作成
pathlib.Path(workdir).mkdir(parents=True, exist_ok=True)
pathlib.Path(tmp_outdir).mkdir(parents=True, exist_ok=True)

# ローカルにコピー（DBFS直参照を避ける）
local_pptx = os.path.join(workdir, os.path.basename(pptx_path))
shutil.copy2(pptx_path, local_pptx)

In [0]:
import os, shutil, subprocess, pathlib, shlex

# 入出力の設定
src_pptx = pptx_path
workdir = "/tmp/unoconv_work"
outdir = "/tmp/unoconv_out"
outfilepath = "/tmp/unoconv_out/databricks_overview_by_ai.pdf"

pathlib.Path(workdir).mkdir(parents=True, exist_ok=True)
pathlib.Path(outdir).mkdir(parents=True, exist_ok=True)

In [0]:
# ローカルにコピー（DBFS直参照を避ける）
local_pptx = os.path.join(workdir, os.path.basename(src_pptx))
shutil.copy2(src_pptx, local_pptx)

# ファイルを変換
res = subprocess.run(
    ["unoconv", "-v", "-f", "pdf", "-o", outfilepath, local_pptx],
    cwd=workdir,
    text=True,
    capture_output=True,
)
print("RC:", res.returncode)
print("--- STDOUT ---\n", res.stdout[:2000])
print("--- STDERR ---\n", res.stderr[:2000])

print("OUTDIR LIST:", os.listdir(outdir))

In [0]:
# できたPDFを元の場所に戻す（必要なら）
for f in os.listdir(outdir):
    if f.lower().endswith("databricks_overview_by_ai.pdf"):
        dbutils.fs.cp(
            "file:" + os.path.join(outdir, f),
            os.path.join(pdf_output_dir, f),
        )

In [0]:
display(dbutils.fs.ls(pdf_output_dir))

## PDF→Images

In [0]:
from pdf2image import convert_from_path

pages = convert_from_path(pdf_output_path)
for i, page in enumerate(pages):
    page.save(f"{png_output_dir}/slide_{i+1}.png", "PNG")

In [0]:
display(dbutils.fs.ls(png_output_dir))

## <参考>画像から AI 関数による情報を取得

In [0]:
sourc_image_path = source_dir
sourc_image_path += "/curated/curated_files"
sourc_image_path += "/746dfa1510e04c260652d56807aded43280608ed4d318351c7e9562ea4184f80"
sourc_image_path += "/png/slide_37.png"

In [0]:
# 画像を表示
import re
from PIL import Image

print("--",sourc_image_path)
im = Image.open(sourc_image_path)
display(im)

In [0]:
from pyspark.sql import functions as F

df = spark.read.format("binaryFile").load(sourc_image_path)
df = df.withColumn(
    "ai_query_output",
    F.expr("""
    ai_query(
        "databricks-llama-4-maverick",
        "これは何ですか？日本語で教えて",
        files => content
    )
    """),
)
df = df.select("ai_query_output")
display(df)

print(f"-- ", sourc_image_path)
print(df.collect()[0]["ai_query_output"])

In [0]:
# end